Импортируем библиотеки

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

 Загружаем данные

In [ ]:
df = pd.read_csv('data/superstore.csv', encoding='latin1')  # часто бывает кодировка
print(f"Размер данных: {df.shape}")
print(f"Колонки: {df.columns.tolist()}")

1. Базовые метрики (как в SQL, но в Pandas)

In [ ]:
total_sales = df['Sales'].sum()
total_profit = df['Profit'].sum()
avg_order_value = df['Sales'].mean()
print(f" Общая выручка: ${total_sales:,.2f}")
print(f" Общая прибыль: ${total_profit:,.2f}")
print(f" Средний чек: ${avg_order_value:.2f}")

2. Топ-5 самых прибыльных категорий

In [ ]:
top_categories = df.groupby('Category')['Profit'].sum().sort_values(ascending=False)
print("Топ категорий по прибыли:")
print(top_categories.head())

3. Анализ временных рядов (продажи по месяцам)

In [ ]:
df['Order Date'] = pd.to_datetime(df['Order Date'], format='%m/%d/%Y')
monthly_sales = df.set_index('Order Date').resample('M')['Sales'].sum()

plt.figure(figsize=(12,5))
monthly_sales.plot(marker='o')
plt.title('Динамика продаж по месяцам')
plt.xlabel('Месяц')
plt.ylabel('Выручка ($)')
plt.grid(True)
plt.tight_layout()
plt.savefig('images/monthly_sales.png')  # Сохраняем график
plt.show()


4. Аналог оконных функций в Pandas - ранжирование товаров внутри категорий

In [ ]:
# Группируем и считаем прибыль по товарам
product_profit = df.groupby(['Category', 'Product Name'])['Profit'].sum().reset_index()

# Добавляем ранк внутри категории
product_profit['rank_in_category'] = product_profit.groupby('Category')['Profit'].rank(method='dense', ascending=False)

# Смотрим топ-3 товара в каждой категории
top_products = product_profit[product_profit['rank_in_category'] <= 3]
print("Топ-3 товара в каждой категории:")
print(top_products.sort_values(['Category', 'rank_in_category']))

5. Правило 20/80 (анализ клиентов)

In [ ]:
customer_sales = df.groupby('Customer ID')['Sales'].sum().sort_values(ascending=False)
customer_sales_cumulative = customer_sales.cumsum() / customer_sales.sum() * 100

# Находим, сколько клиентов дают 80% выручки
top_customers_count = (customer_sales_cumulative <= 80).sum()
print(f"\Правило 20/80: {top_customers_count} клиентов ({top_customers_count/len(customer_sales)*100:.1f}%) дают 80% выручки")

6. Красивый дашборд из 4 графиков

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))


In [ ]:
# График 1: Прибыль по категориям
category_profit = df.groupby('Category')['Profit'].sum().sort_values()
axes[0,0].barh(category_profit.index, category_profit.values, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
axes[0,0].set_title('Прибыль по категориям')
axes[0,0].set_xlabel('Прибыль ($)')

In [ ]:
# График 2: Динамика прибыли по месяцам (линейный)
monthly_profit = df.set_index('Order Date').resample('M')['Profit'].sum()
axes[0,1].plot(monthly_profit.index, monthly_profit.values, marker='o', color='#95E1D3', linewidth=2)
axes[0,1].fill_between(monthly_profit.index, 0, monthly_profit.values, alpha=0.3, color='#95E1D3')
axes[0,1].set_title('Динамика прибыли по месяцам')
axes[0,1].set_xlabel('Месяц')
axes[0,1].set_ylabel('Прибыль ($)')
axes[0,1].grid(True, alpha=0.3)

In [ ]:
# График 3: Корреляция между скидкой и прибылью (scatter plot)
axes[1,0].scatter(df['Discount'], df['Profit'], alpha=0.5, color='#F38181', s=20)
axes[1,0].set_title('Влияние скидки на прибыль')
axes[1,0].set_xlabel('Скидка (%)')
axes[1,0].set_ylabel('Прибыль ($)')
axes[1,0].axhline(y=0, color='gray', linestyle='--', alpha=0.5)

In [ ]:
# График 4: Продажи по регионам (pie chart)
region_sales = df.groupby('Region')['Sales'].sum()
colors = ['#FFD93D', '#6BCF7F', '#4D96FF', '#FF6B6B']
axes[1,1].pie(region_sales.values, labels=region_sales.index, autopct='%1.1f%%', 
              colors=colors, startangle=90)
axes[1,1].set_title('Доля продаж по регионам')

plt.suptitle('Анализ продаж супермаркета', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('images/dashboard.png', dpi=100, bbox_inches='tight')
plt.show()

7. Формулируем бизнес-выводы 

In [ ]:
# Вывод 1: Самая прибыльная категория
best_category = category_profit.idxmax()
best_category_profit = category_profit.max()
print(f"1.Самая прибыльная категория: {best_category} (${best_category_profit:,.0f})")


# Вывод 2: Влияние скидок
high_discount = df[df['Discount'] > 0.2]  # скидки больше 20%
if len(high_discount) > 0:
    avg_profit_with_discount = high_discount['Profit'].mean()
    print(f"2.Средняя прибыль при скидках >20%: ${avg_profit_with_discount:.2f}")


# Вывод 3: Сезонность
best_month = monthly_sales.idxmax().strftime('%B %Y')
best_month_sales = monthly_sales.max()
print(f"3.Лучший месяц: {best_month} (${best_month_sales:,.0f})")


# Вывод 4: Лояльные клиенты
repeat_customers = df['Customer ID'].value_counts()
loyal_customers = repeat_customers[repeat_customers > 1].count()
print(f"4.Лояльных клиентов (>1 покупки): {loyal_customers}")
